In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r'data/Ball_By_Ball.csv')

In [3]:
df.head()

,MatcH_id,Over_id,Ball_id,Innings_No,Team_Batting,Team_Bowling,Striker_Batting_Position,Extra_Type,Runs_Scored,Extra_runs,...,Fielder_match_SK,Fielder_SK,Bowler_match_SK,BOWLER_SK,PlayerOut_match_SK,BattingTeam_SK,BowlingTeam_SK,Keeper_Catch,Player_out_sk,MatchDateSK
0,598028,15,6,1,5,2,6.0,No Extras,4,0,...,-1,-1,20343,82,-1,4,1,0,0,20130420
1,598028,14,1,1,5,2,5.0,No Extras,1,0,...,-1,-1,20348,345,-1,4,1,0,0,20130420
2,598028,14,2,1,5,2,3.0,No Extras,1,0,...,-1,-1,20348,345,-1,4,1,0,0,20130420
3,598028,14,3,1,5,2,5.0,No Extras,1,0,...,-1,-1,20348,345,-1,4,1,0,0,20130420
4,598028,14,4,1,5,2,3.0,No Extras,0,0,...,-1,-1,20348,345,-1,4,1,0,0,20130420


In [14]:
df.shape[0]

150451

In [5]:
df = df.sort_values(['MatcH_id', 'Innings_No', 'Over_id', 'Ball_id']).reset_index(drop=True)
df.head()

,MatcH_id,Over_id,Ball_id,Innings_No,Team_Batting,Team_Bowling,Striker_Batting_Position,Extra_Type,Runs_Scored,Extra_runs,...,Fielder_match_SK,Fielder_SK,Bowler_match_SK,BOWLER_SK,PlayerOut_match_SK,BattingTeam_SK,BowlingTeam_SK,Keeper_Catch,Player_out_sk,MatchDateSK
0,335987,1,1,1,1,2,1.0,legbyes,0,1,...,-1,-1,12702,13,-1,0,1,0,0,20080418
1,335987,1,2,1,1,2,2.0,No Extras,0,0,...,-1,-1,12702,13,-1,0,1,0,0,20080418
2,335987,1,3,1,1,2,2.0,wides,0,1,...,-1,-1,12702,13,-1,0,1,0,0,20080418
3,335987,1,4,1,1,2,2.0,No Extras,0,0,...,-1,-1,12702,13,-1,0,1,0,0,20080418
4,335987,1,5,1,1,2,2.0,No Extras,0,0,...,-1,-1,12702,13,-1,0,1,0,0,20080418


In [25]:
df.Over_id.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])

In [6]:
import json
import time
import pandas as pd
from azure.eventhub import EventHubProducerClient, EventData

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()  # loads .env from current directory

EVENT_HUB_CONNECTION_STRING = os.getenv("EVENT_HUB_CONNECTION_STRING")
EVENT_HUB_NAME = os.getenv('EVENT_HUB_NAME')

In [ ]:
# Creating producer client

producer = EventHubProducerClient.from_connection_string(
    conn_str=EVENT_HUB_CONNECTION_STRING,
    eventhub_name=EVENT_HUB_NAME
)

In [27]:
for n in range(5):
    payload = df.iloc[n].to_dict()
    event_json = json.dumps(payload)
    # print(event_json)
    print(f"Sent event: Match {payload['MatcH_id']} | Over id {payload['Over_id']}, Ball id {payload['Ball_id']}")


Sent event: Match 335987 | Over id 1, Ball id 1
Sent event: Match 335987 | Over id 1, Ball id 2
Sent event: Match 335987 | Over id 1, Ball id 3
Sent event: Match 335987 | Over id 1, Ball id 4
Sent event: Match 335987 | Over id 1, Ball id 5


In [29]:
with producer:
    event_batch = producer.create_batch()
    
    for i in range(5): #df.shape[0]):
        payload = df.iloc[n].to_dict()
        event_json = json.dumps(payload)

        try:
            event_batch.add(EventData(event_json))
        except ValueError:
            producer.send_batch(event_batch)
            event_batch = producer.create_batch()
            event_batch.add(EventData(event_json))

        print(f"Sent event: Match {payload['MatcH_id']} | Over id {payload['Over_id']}, Ball id {payload['Ball_id']}")
        time.sleep(3)

    if len(event_batch) > 0:
        producer.send_batch(event_batch)

print("IPL streaming completed successfully.")

Sent event: Match 335987 | Over id 1, Ball id 5
Sent event: Match 335987 | Over id 1, Ball id 5
Sent event: Match 335987 | Over id 1, Ball id 5
Sent event: Match 335987 | Over id 1, Ball id 5
Sent event: Match 335987 | Over id 1, Ball id 5
IPL streaming completed successfully.
